In [1]:
import warnings
warnings.filterwarnings("ignore")
from keras.engine.topology import Layer
from keras.layers import Bidirectional, Concatenate, Permute, Dot, Input, LSTM, GRU, Multiply, Reshape
from keras.layers import RepeatVector, Dense, Activation, Lambda, Embedding
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.models import load_model, Model
import keras.backend as K
import keras
import numpy as np
import random
import tqdm
import matplotlib.pyplot as plt
from keras.utils import to_categorical
import pandas as pd
# from tcn import TCN
%matplotlib inline
from keras.layers import merge, Input, Dense, TimeDistributed, Lambda                                   
import keras
from keras.models import Sequential, Model
from keras.layers import Conv2D, GlobalMaxPooling2D,MaxPooling2D, Flatten, Dense, BatchNormalization, Activation, Input, GlobalAveragePooling2D,AveragePooling2D, Add
from keras.optimizers import Adam, rmsprop
from keras.callbacks import EarlyStopping
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasRegressor
from keras.constraints import maxnorm
MAX_SENT_LENGTH = 55
MAX_SENTS = 24
VALIDATION_SPLIT = 0.2

Using TensorFlow backend.


In [2]:
from sklearn.preprocessing import MinMaxScaler  
from sklearn.preprocessing import StandardScaler  
from pandas import concat

std = StandardScaler()

In [3]:
import pandas as pd
#matplotlib inline
#参数初始化
discfile = '../data/data_wea_time_unify.csv'

data = pd.read_csv(discfile)

In [4]:
futrue_time=24

time_step=24

In [5]:
for i in range(24,96):
    data['PM25_m'+str(i)]=data['PM25_mean'].shift(-i)
data=data.dropna(how='any')

In [6]:
target_data=data.iloc[:,76:]
source_data=data.iloc[:,4:58]

import copy
sd=copy.deepcopy(source_data)
td=copy.deepcopy(target_data)

td=td.dropna(how='any')
# td=std.fit_transform(td)
sd=std.fit_transform(sd)
# sd=sd.values
td=td.values
td_test=td[5801:,:futrue_time]   
td=td[:5801,:futrue_time]
sd_m=[]
for i in range(1,MAX_SENTS+1):
    locals()['sd'+str(i)]=sd[i-1:5800+i]

    sd_m.append(locals()['sd'+str(i)])
sd_m=(np.array(sd_m).swapaxes(0,1))


######test
sd_m_test=[]
for i in range(1,25):
    locals()['sd_t'+str(i)]=sd[5800+i:5969-24+i]
    
    sd_m_test.append(locals()['sd_t'+str(i)])

sd_m_test=(np.array(sd_m_test).swapaxes(0,1))

In [7]:
wea_data=data.iloc[:,67:75]
# wea_data=std.fit_transform(wea_data)
# wea_data=pd.DataFrame(wea_data)
w=wea_data[['Conditions','Wind Dir','Quality evaluation']]
# w=std.fit_transform(w)
w_m=[]
for i in range(1,25):
    locals()['w'+str(i)]=w[i-1:5800+i].values
    
    w_m.append(locals()['w'+str(i)])
    
wea=wea_data[['Dew Point','Humidity','Pressure','Temp.','Wind Speed']]
wea=std.fit_transform(wea)
wea_m=[]
for i in range(1,25):
    locals()['wea'+str(i)]=wea[i-1:5800+i]
    
    wea_m.append(locals()['wea'+str(i)])

In [8]:
time_data=data.iloc[:,64:67]
# time_data=std.fit_transform(time_data)
time_m=[]
for i in range(1,25):
    locals()['time'+str(i)]=time_data[i-1:5800+i].values
    
    time_m.append(locals()['time'+str(i)])

In [9]:
time_m=(np.array(time_m).swapaxes(0,1))
w_m=(np.array(w_m).swapaxes(0,1))
wea_m=(np.array(wea_m).swapaxes(0,1))

In [10]:
time_m_test=[]
for i in range(1,25):
    locals()['time_t'+str(i)]=time_data[5800+i:5969-24+i].values
    
    time_m_test.append(locals()['time_t'+str(i)])

time_m_test=(np.array(time_m_test).swapaxes(0,1))

w_m_test=[]
for i in range(1,25):
    locals()['w_t'+str(i)]=w[5800+i:5969-24+i].values
    
    w_m_test.append(locals()['w_t'+str(i)])

w_m_test=(np.array(w_m_test).swapaxes(0,1))

wea_m_test=[]
for i in range(1,25):
    locals()['wea_t'+str(i)]=wea[5800+i:5969-24+i]
    
    wea_m_test.append(locals()['wea_t'+str(i)])

wea_m_test=(np.array(wea_m_test).swapaxes(0,1))